In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report

In [12]:
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (width, height, depth)
    
    model.add(Conv2D(32, (5,5), padding='same', input_shape=inputShape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(classes, activation='softmax'))
    
    return model

In [15]:
LR = 1e-3
EPOCHS = 10
BS = 128
(trainX,trainy), (testX, testy) = mnist.load_data()

In [18]:
trainX=trainX.reshape(trainX.shape[0], 28,28,1)
testX=testX.reshape(testX.shape[0], 28,28,1)

In [19]:
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

In [20]:
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

In [21]:
opt = Adam(lr=LR)
model = build(28,28,1,10)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
h = model.fit(trainX,trainy, validation_data=(testX,testy), batch_size=BS, epochs=EPOCHS, verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 52s 860us/sample - loss: 0.7111 - accuracy: 0.7637 - val_loss: 0.1104 - val_accuracy: 0.9691
Epoch 2/10
60000/60000 [==============================] - 58s 964us/sample - loss: 0.2615 - accuracy: 0.9223 - val_loss: 0.0705 - val_accuracy: 0.9802
Epoch 3/10
60000/60000 [==============================] - 54s 900us/sample - loss: 0.1925 - accuracy: 0.9453 - val_loss: 0.0519 - val_accuracy: 0.9854
Epoch 4/10
60000/60000 [==============================] - 48s 798us/sample - loss: 0.1572 - accuracy: 0.9548 - val_loss: 0.0459 - val_accuracy: 0.9861
Epoch 5/10
60000/60000 [==============================] - 49s 812us/sample - loss: 0.1368 - accuracy: 0.9617 - val_loss: 0.0393 - val_accuracy: 0.9873
Epoch 6/10
60000/60000 [==============================] - 49s 809us/sample - loss: 0.1231 - accuracy: 0.9650 - val_loss: 0.0413 - val_accuracy: 0.9875
Epoch 7/10
60000/60000 [====================

In [26]:
predictions = model.predict(testX)
print(classification_report(
    testy.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=[str(x) for x in lb.classes_]))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      1.00      1135
           2       0.99      1.00      0.99      1032
           3       0.99      1.00      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.98      0.99      0.98       892
           6       1.00      0.98      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.98      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [27]:
model.save('digit_weights.h5')